In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, LlamaForCausalLM, LlamaTokenizer
from torch.utils.data import Dataset, DataLoader
from textbrewer import GeneralDistiller, TrainingConfig, DistillationConfig
import os
from torch.optim import AdamW
from torch.utils.data import Dataset
import json

In [ ]:
class KnowledgeDataset(Dataset):
    #filepath=r"D:\models\training_data"
    def __init__(self, filepath, tokenizer, max_length=128):
        with open(filepath, "r", encoding="utf-8") as f:
            self.lines = [line.strip() for line in f if line.strip()]
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.lines)

    def __getitem__(self, idx):
        text = self.lines[idx]
        enc = self.tokenizer(
            text,
            return_tensors="pt",
            max_length=self.max_length,
            padding="max_length",
            truncation=True
        )
        input_ids = enc["input_ids"].squeeze()
        attention_mask = enc["attention_mask"].squeeze()
        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": input_ids.clone()
        }


In [ ]:
#device = "cuda"   You can change to 'cuda' if GPU available

teacher_path =r"D:\models\llama8b"
teacher = AutoModelForCausalLM.from_pretrained(
    teacher_path,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)
# teacher = AutoModelForCausalLM.from_pretrained(teacher_path).to(device)
tokenizer = AutoTokenizer.from_pretrained(teacher_path)


In [ ]:
student_path =  r"D:\models\llama3b"
student = AutoModelForCausalLM.from_pretrained(
    student_path,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

In [ ]:

prompt_template = """You are a helpful assistant. return awb number, :
❗ Important:
- Do **not** use given examples for any fields
- Do **not** use any parts from the prompt as fields
- Only return the final JSON object.
- Do **not** add any explanation, markdown formatting, or code.
- Do **not** include backticks (```) or language tags like ```json.
- Do **not** generate Python or any other code.
- If data is missing, return `null`, but do not fabricate.
- Output must be valid and clean JSON.
- Only take values from the given mail
 
 
 
 
Subject: {subject}
From: {from_}
To: {to}
Body:{body}
######################################################
 You are assisting the Airline Cargo team in extracting specific business-critical entities from customer emails. Each email consists of a subject and body, both delimited by triple backticks (```).
 
You must return a JSON array of dictionaries, each representing one AWB (Air Waybill) entry with its corresponding entities. The structure should follow these rules:
Return a JSON array of dictionaries.
Each dictionary must include the extracted AWB and its associated entities.
Omit any fields not found in the email; DO NOT assume values.
 
The final output must be pure JSON: no explanations, no backticks, no extra text.
The json output should appear as per the following format
[
    {{
        "AWB": "",
        "FlightNo": "",
        "Departure-date": "",
        "total-pieces": ,
        "pieces@dimensions": [""],
        "dimension-unit": [""],
        "Weight": ,
        "weight-unit": "",
        "special-instruction": "",
        "commodity-description": "",
        "product-code": "",
        "Source": "",
        "Destination": ""
    }}
]
AWB (Air Waybill):
Must be 11-digit numbers starting with valid airline prefixes: <AWB_PREFIX>.
May be referred to as "MAWB" or "GUIA".
Remove hyphens or spaces.
One dictionary per AWB; multiple AWBs = multiple dictionaries.
 
FlightNo:
Must start with valid carrier codes: <AIRLINE_PREFIX>.
Format: airline code + number
do not take the date value for the flight number if there is flight date attached with flight number (eg KE706/18APR in this only take KE706)
if no values are found keep as null
 
Departure-date:
Extract in YYYY-MM-DD format.
If given as a range like 23/24/07, choose the latest date (i.e., 2025-07-24).
If given as a relative day (e.g., "next Monday"), assume today's date is 2025-03-22 (Saturday) and resolve accordingly.
 
total-pieces:
Integer value representing total cargo pieces.
 
pieces@dimensions:
Format: list like ["2@24x17x9"].
May appear as pcs x l x b x h or pcs @ l x b x h.
Extract all combinations; prioritize individual dimensions over total.
 
dimension-unit:
Supported units: "CM", "M", "IN", "OTH".
Provide as a list matching the sequence of pieces@dimensions.
 
Weight:
If individual weights are given, compute the total.
Use chargeable weight (CW) or gross weight (G/W) if explicitly mentioned.
If weight is embedded in piece-dimension combos, extract accordingly.
 
weight-unit:
Supported values: "KG", "KGS", "LBS", "OTH".
 
special-instruction:
Extract any special handling notes.
Always translate to English.
 
commodity-description:
Free text describing the goods.
Always translate to English.
 
product-code:
If not explicitly given, infer from commodity description:
"GEN" for general cargo
"HAZ" for hazardous materials
"DG" for dangerous goods
 
Source / Destination:
Extract from IATA codes in formats like EWR-OME (EWR = Source, OME = Destination).
Do not assume source location from sender’s location or from flight number.
 
Must Translate all extracted text into  json.
Do not fabricate missing values.
Always return a clean JSON output only — no markdown, no backticks, no wrapping text.
Must Not generate anything other than the base json file. Do NOT generate any code.
Only the output is required do not generate anything else
 
since there is a json format given generate just as the json format. Do not generate in loop. if it start to generate in loop stop the generation.
 
"""

     

In [ ]:
class PromptResponseDataset(Dataset):
    def __init__(self, file_path, tokenizer, prompt_template, max_length=512):
        self.tokenizer = tokenizer
        self.prompt_template = prompt_template  # <-- store the prompt
        self.max_length = max_length
        self.samples = []

        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)

        for item in data:
            subject = item.get("subject", "")
            from_ = item.get("from", "")
            to = item.get("to", "")
            body = item.get("body", "")

            self.samples.append({
                "subject": subject,
                "from_": from_,
                "to": to,
                "body": body
            })

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        sample = self.samples[idx]
        subject = sample["subject"]
        from_ = sample["from_"]
        to = sample["to"]
        body = sample["body"]

        # Use the shared prompt template
        prompt = self.prompt_template.format(subject=subject, from_=from_, to=to, body=body)

        inputs = self.tokenizer(
            prompt,
            return_tensors='pt',
            max_length=self.max_length,
            truncation=True,
            padding='max_length'
        )

        return {
            "input_ids": inputs["input_ids"].squeeze(0),
            "attention_mask": inputs["attention_mask"].squeeze(0),
            "labels": inputs["input_ids"].squeeze(0).clone(),
            "prompt": prompt,
            "subject": subject,
            "from_": from_,
            "to": to,
            "body": body
        }


In [ ]:
file_path = r"D:\distil\structured_outputs.json"
dataset = PromptResponseDataset(file_path, tokenizer, prompt_template)

train_loader = torch.utils.data.DataLoader(dataset, batch_size=2, shuffle=True)
# dataset = KnowledgeDataset(r"C:\Users\211369\Desktop\program\distil\airline.txt", tokenizer)
# train_dataloader = DataLoader(dataset,batch_size=2,shuffle=True)        

In [ ]:
from torch.utils.data import DataLoader

dataloader = DataLoader(dataset, batch_size=1, shuffle=True)
tokenizer.pad_token = tokenizer.eos_token

for batch in dataloader:
    prompts = batch["prompt"]  # batch["prompt"] is a list of strings
    for prompt in prompts:
        print("📨 Prompt:\n", prompt)
    break



In [ ]:
# def generate_response(prompt: str, max_tokens=512):
#     inputs = tokenizer(prompt, return_tensors="pt", truncation=True)
#     with torch.no_grad():
#         outputs = teacher.generate(**inputs, max_new_tokens=max_tokens)
#     return tokenizer.decode(outputs[0], skip_special_tokens=True)

# response = generate_response(prompt)
# generated_text = response[len(prompt):].strip()

# print(generated_text)

In [ ]:
tokenizer.pad_token = tokenizer.eos_token
batch = next(iter(train_loader))
with torch.no_grad():
    t_out = teacher(**batch)
s_out = student(**batch)
print(t_out.logits.shape)  # [B, T, V]
print(s_out.logits.shape)

In [ ]:
optimizer = AdamW(student.parameters(), lr= 5e-5)

train_config = TrainingConfig(
    device='cuda',                  # or 'cpu' if no GPU
    output_dir='./saved_model',
    log_dir='./log',
)

distill_config = DistillationConfig(
    temperature=2.0,
    kd_loss_type='ce',              # 'ce' for KLDiv, 'mse' for regression
    kd_loss_weight=1.0,             # use 1.0 for pure distillation
    hard_label_weight=1.0,          # set 0.0 to ignore ground-truth
)

In [ ]:

def get_adaptor(batch, model_output):
    return {
        "logits": model_output.logits,
        "labels": batch.get("labels", None) 
    }


def batch_postprocessor(batch):
    return {
        "input_ids": batch["input_ids"].to(device),
        "attention_mask": batch["attention_mask"].to(device),
        "labels": batch["labels"].to(device),
    }
def callback(step=None, loss=None, lr=None, model=None):
    if loss is not None:
        print(f"[Step {step}] Total Loss: {loss:.4f} | LR: {lr:.6f}")
    else:
        print(f"[Step {step}] Loss is None")



In [ ]:
tokenizer.pad_token = tokenizer.eos_token


distiller = GeneralDistiller(
    train_config=train_config,
    distill_config=distill_config,
    model_T=teacher,
    model_S=student,
    adaptor_T=get_adaptor,
    adaptor_S=get_adaptor,
    
)


In [ ]:
print(distiller.__class__)


In [ ]:
batch = next(iter(train_loader))
loss, loss_dict = distiller.train_on_batch(batch, {})
print("Loss:", loss)


In [ ]:
tokenizer.pad_token = tokenizer.eos_token
print(distiller.__class__)
from types import MethodType

def force_general_train(self, *args, **kwargs):
    print("✅ This is GeneralDistiller.train()")
    return GeneralDistiller.train(self, *args, **kwargs)

distiller.train = MethodType(force_general_train, distiller)

original_train_on_batch = distiller.train_on_batch

def wrapped_train_on_batch(self, batch, args):
    loss, loss_dict = original_train_on_batch(batch, args)
    print(f"[DISTILL STEP] Loss: {loss.item():.4f}")
    return loss, loss_dict

from types import MethodType
distiller.train_on_batch = MethodType(wrapped_train_on_batch, distiller)

distiller.train_with_num_epochs(
    optimizer=optimizer,
    scheduler=None,
    tqdm_disable=False,
    dataloader=train_loader,
    max_grad_norm=1.0,
    num_epochs=5,
    callback=callback,
    batch_postprocessor=None
)   

In [ ]:
distilled_model=r"D:\models\op"

student.save_pretrained(distilled_model)
tokenizer.save_pretrained(distilled_model)